### Tensor Flow
TensorFlow is a Google's library that expresses everything into graph (expression/variables/operation).

In [18]:
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data

###### PLACEHOLDER

We will use data from MNIST example data from tensorflow.
Input Dataset: 28px x 28px 
Output Label: 10

In [19]:
x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 10])

Let's define operations in terms of expression and variable to be used in tenfow session below.

###### FEED FORWARD

We will be creat a neural network with 3 (input, hidden and output) layers. Therefore, we need to randomly initialize weights and biasis for 2 synapsis:
- 1st synapsis connecting input layer and hidden layer.
- 2nd synapsis connecting hidden layer and output layer.

Then, we will calculate the weighted sum for 1st synapsis and pass it through 'relu' activation function and then we will calculate the weighted sum of 2nd synapsis and pass it through softmax function to get the predictions.

In [20]:
i2h_weights = tf.Variable(tf.random_normal([784, 300], stddev=0.03), name='InputToHiddenWeights')
i2h_bias = tf.Variable(tf.random_normal([300]), name='InputToHiddenBias')

h2o_weights = tf.Variable(tf.random_normal([300, 10], stddev=0.03), name='HiddenToOutputWeights')
h2o_bias = tf.Variable(tf.random_normal([10]), name='HiddenToOutputBias')

i2h_weighted_sum = tf.add(tf.matmul(x, i2h_weights), i2h_bias)
i2h_activated_weighted_sum = tf.nn.relu(i2h_weighted_sum)

predicted_y = tf.nn.softmax(tf.add(tf.matmul(i2h_activated_weighted_sum, h2o_weights), h2o_bias)) 

###### COST/LOSS FUNCTION 

In [21]:
scaled_y = tf.clip_by_value(predicted_y, 1e-10, 0.9999999)
cost_function = -tf.reduce_mean(tf.reduce_sum(y * tf.log(scaled_y) + (1 - y) * tf.log(1 - scaled_y), axis=1))

###### GRADIENT DESCENT

In [22]:
gradient_descent = tf.train.GradientDescentOptimizer(learning_rate=0.5).minimize(cost_function)

###### ACCURACY

In [23]:
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(predicted_y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

We need to setup an object that will initialize all the afore-defined variables, operations and expression to be run inside tensorflow session below. 

In [24]:
initializer = tf.global_variables_initializer()

###### TRAINING
In tensorflow, all the operation happens inside tensorflow session. 

All the variables (i2h_weights, i2h_bias, h2o_weights and h2o_bias) and operation (gradient_descent, cost_function, accuracy, initializer, i2h_weighted_sum and i2h_activated_weighted_sum) defined above are run inside.

NOTE: We will be using mini-batch gradient descent instead of batch descent for better performace.

In [28]:
#load the tensorflow's example dataset
mnist_dataset = input_data.read_data_sets("MNIST_data/", one_hot=True)

#tensor flow session begins
with tf.Session() as SESSION:
    #initializes all the global variables and operation defined previously
    SESSION.run(initializer)
    
    #number of mini-batch (each batch of size: 100)
    no_100_batch = int(len(mnist_dataset.train.labels) / 100)
    
    #loopr for 10 epoch
    for iteration in range(10):
        #loop through each mini-batch
        for i in range(iteration):
            #extract x_ and y_
            x_, y_ = mnist_dataset.train.next_batch(batch_size=100)
            #feed extract x_ and y_ into placeholder x and y
            #and calculate the cost after each optimization with gradient descent
            _, cost = SESSION.run([gradient_descent, cost_function], feed_dict={x: x_, y: y_})
        
        
        print("COST: %f " % (cost / no_100_batch))
        
    print(SESSION.run(accuracy, feed_dict={x: mnist_dataset.test.images, y: mnist_dataset.test.labels}))

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
COST: 0.756044 
COST: 0.264363 
COST: 0.198879 
COST: 0.166144 
COST: 0.141082 
COST: 0.123921 
COST: 0.105480 
COST: 0.093716 
COST: 0.084787 
COST: 0.074629 
0.9733


That's how tensorflow rolls!